### Load cleaned data and pre-defined classes

In [9]:
from data_clean import Data_to_Clean,Data_to_Analyze
import pickle
import pandas as pd
MBTI_types=[
    "ISTJ", "ISFJ", "INFJ", "INTJ",
    "ISTP", "ISFP", "INFP", "INTP",
    "ESTP", "ESFP", "ENFP", "ENTP",
    "ESTJ", "ESFJ", "ENFJ", "ENTJ"
]
cleaned_data=dict.fromkeys(MBTI_types, None)
for type in cleaned_data.keys():
    file_path=f"Data\\cleaned_data\\{type}_cleaned.pkl" 
    try:
        with open(file_path, 'rb') as f:
            cleaned_data[type] = pickle.load(f)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except pickle.UnpicklingError:
        print(f"Error: Could not unpickle the file {file_path}. It might be corrupted or not a valid pickle file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [10]:
enfp=cleaned_data["ENFP"]
enfp.basic_identities

type                                                                  ENFP
sentence_quantity        [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 5...
ave_sentence_quantity                                            48.546667
word_count               [1856, 1508, 1359, 1308, 1145, 1366, 1111, 927...
ave_word_count                                                 1345.533333
upper_ratio              [0.03661812526509261, 0.04079665917121748, 0.0...
ave_upper_ratio                                                   0.056934
reading_ease             [82.34, 68.87, 74.9, 75.5, 69.38, 70.02, 69.28...
ave_reading_ease                                                 74.426548
GF_index                 [7.16, 7.96, 7.57, 7.35, 7.49, 9.48, 7.57, 9.5...
ave_GF_index                                                      7.477807
dtype: object

In [11]:
enfp.data

,type,posts
0,ENFP,"[[want, go, trip, without, stay, behind, would..."
1,ENFP,"[[hey, enfps, post, thread, philosophy, board,..."
2,ENFP,"[[brother, far, right], [im, infj, lot, see, f..."
3,ENFP,"[[learn, say, right, way, great, talent, yet, ..."
4,ENFP,"[[know, right, year, old, fucking, crazy, shit..."
...,...,...
670,ENFP,"[[thank, look, forward, connect, others, forum..."
671,ENFP,"[[seriously, relate, struggle, one, big, hurdl..."
672,ENFP,"[[hey, happy, meet, get, idea, great, quality,..."
673,ENFP,"[[leoni, really, bad, social, anxiety, take, m..."
